In [1]:
import os
from dotenv import load_dotenv

In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [9]:
from pinecone import Pinecone, ServerlessSpec

In [4]:
load_dotenv()

True

In [5]:
api_key = os.getenv("API_KEY")
db_host = os.getenv("DB_HOST")
db_password = os.getenv("DB_PASSWORD")
db_user = os.getenv("DB_USER")
db_name = os.getenv("DB_NAME")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [14]:
index_name = "haderach"

In [7]:
pc = Pinecone(api_key=pinecone_api_key)

In [16]:
pc.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [18]:
pc.delete_index(name=index_name)

In [ ]:
import vanna
from vanna.remote import VannaDefault

In [6]:
model_name = 'al-gaib'
vn = VannaDefault(model=model_name, api_key=api_key)

In [7]:
vn.connect_to_postgres(host=db_host,
                       dbname=db_name,
                       user=db_user,
                       password=db_password,
                       port=5432)

In [8]:
# df_information_schema = vn.run_sql('''SELECT *
#                                     FROM information_schema.columns
#                                     WHERE table_schema != 'public'; ''')
# df_information_schema.head()

In [21]:
plan = vn.get_training_plan_generic(df_information_schema)

In [22]:
for table_name in plan.get_summary():
  print(table_name[len("Train on Information Schema: "):])

quizzard.pg_catalog pg_proc
quizzard.pg_catalog pg_type
quizzard.pg_catalog pg_attribute
quizzard.pg_catalog pg_class
quizzard.pg_catalog pg_attrdef
quizzard.pg_catalog pg_constraint
quizzard.pg_catalog pg_inherits
quizzard.pg_catalog pg_index
quizzard.pg_catalog pg_operator
quizzard.pg_catalog pg_opfamily
quizzard.pg_catalog pg_opclass
quizzard.pg_catalog pg_am
quizzard.pg_catalog pg_amop
quizzard.pg_catalog pg_amproc
quizzard.pg_catalog pg_language
quizzard.pg_catalog pg_largeobject_metadata
quizzard.pg_catalog pg_largeobject
quizzard.pg_catalog pg_aggregate
quizzard.pg_catalog pg_statistic
quizzard.pg_catalog pg_statistic_ext
quizzard.pg_catalog pg_statistic_ext_data
quizzard.pg_catalog pg_rewrite
quizzard.pg_catalog pg_trigger
quizzard.pg_catalog pg_event_trigger
quizzard.pg_catalog pg_description
quizzard.pg_catalog pg_cast
quizzard.pg_catalog pg_enum
quizzard.pg_catalog pg_namespace
quizzard.pg_catalog pg_conversion
quizzard.pg_catalog pg_depend
quizzard.pg_catalog pg_database
qu

In [23]:
vn.train(plan=plan)

In [ ]:
# DDL = '''
#     CREATE TABLE IF NOT EXISTS public."Answers" (
#         "Id" uuid NOT NULL DEFAULT gen_random_uuid(),
#         "CreatedAt" timestamptz NOT NULL,
#         "LastUpdatedAt" timestamptz NOT NULL,
#         "OptionId" uuid NOT NULL,
#         "QuestionId" uuid NOT NULL,
#         "QuizSessionId" uuid NOT NULL,
#         "UserId" uuid NOT NULL,
#         CONSTRAINT "Answers_pkey" PRIMARY KEY ("Id"),
#         CONSTRAINT "Answers_OptionId_fkey" FOREIGN KEY ("OptionId") 
#             REFERENCES public."Options" ("Id") ON DELETE CASCADE,
#         CONSTRAINT "Answers_QuestionId_fkey" FOREIGN KEY ("QuestionId") 
#             REFERENCES public."Questions" ("Id") ON DELETE CASCADE,
#         CONSTRAINT "Answers_QuizSessionId_fkey" FOREIGN KEY ("QuizSessionId") 
#             REFERENCES public."QuizSessions" ("Id") ON DELETE CASCADE,
#         CONSTRAINT "Answers_UserId_fkey" FOREIGN KEY ("UserId") 
#             REFERENCES public."Users" ("Id") ON DELETE CASCADE
#     );
# '''

In [16]:
DDL = '''
CREATE TABLE IF NOT EXISTS public."Answers" (
    "Id" uuid PRIMARY KEY DEFAULT gen_random_uuid(),
    "CreatedAt" timestamptz NOT NULL,
    "LastUpdatedAt" timestamptz NOT NULL,
    "OptionId" uuid NOT NULL REFERENCES public."Options" ("Id") ON DELETE CASCADE,
    "QuestionId" uuid NOT NULL REFERENCES public."Questions" ("Id") ON DELETE CASCADE,
    "QuizSessionId" uuid NOT NULL REFERENCES public."QuizSessions" ("Id") ON DELETE CASCADE,
    "UserId" uuid NOT NULL REFERENCES public."Users" ("Id") ON DELETE CASCADE
);
'''

In [17]:
vn.train(ddl=DDL)

Adding ddl: 
CREATE TABLE IF NOT EXISTS public."Answers" (
    "Id" uuid PRIMARY KEY DEFAULT gen_random_uuid(),
    "CreatedAt" timestamptz NOT NULL,
    "LastUpdatedAt" timestamptz NOT NULL,
    "OptionId" uuid NOT NULL REFERENCES public."Options" ("Id") ON DELETE CASCADE,
    "QuestionId" uuid NOT NULL REFERENCES public."Questions" ("Id") ON DELETE CASCADE,
    "QuizSessionId" uuid NOT NULL REFERENCES public."QuizSessions" ("Id") ON DELETE CASCADE,
    "UserId" uuid NOT NULL REFERENCES public."Users" ("Id") ON DELETE CASCADE
);



'352494-ddl'

In [23]:
vn.train(documentation="When querying any table, use the fully qualified table name including schema")

Adding documentation....


'2438295-doc'

In [21]:
sql_queries = [
    ''' SELECT 
    "QuizSessionId",
    COUNT(*) AS answers_in_session,
    MIN("CreatedAt") AS session_start,
    MAX("CreatedAt") AS session_end,
    COUNT(DISTINCT "UserId") AS unique_users
    FROM public."Answers"
    GROUP BY "QuizSessionId"
    ORDER BY session_start;
    ''',

    ''' SELECT 
    q."QuestionText",
    COUNT(a.*) AS times_answered,
    COUNT(DISTINCT a."UserId") AS unique_users_answered
    FROM public."Answers" a
    JOIN public."Questions" q ON q."Id" = a."QuestionId"
    GROUP BY a."QuestionId", q."QuestionText"
    ORDER BY times_answered DESC;
    ''',

    ''' SELECT 
    CONCAT(u."FirstName", ' ', u."LastName") AS FullName, 
    COUNT(q."Score") AS points,
    TO_CHAR(q."EndTime" - q."StartTime", 'HH24:MI:SS') AS duration
    FROM public."Users" AS u
    JOIN public."QuizSessions" AS q ON u."Id" = q."UserId"
    GROUP BY FullName, duration
    ORDER BY points DESC;
    ''',

    ''' SELECT 
    CONCAT("FirstName", ' ', "LastName") AS FullName,
    "Email"
    FROM public."Users"
    WHERE "EmailConfirmed" = true;
    '''
]

In [ ]:
vn.run_sql(sql_queries[1])

In [22]:
vn.run_sql(sql_queries[3]).head()

,fullname,Email
0,Admin Oga,admin@admin.com
1,Christian Ichebi,christianichebi@gmail.com
2,Zee Bola,zajibola41@gmail.com
3,Odunayo Bamigboye,odunayo845@gmail.com
4,eka essien,ekaessien@gmail.com


In [21]:
vn.train(
        question="What are the most popular questions being answered, and how many unique users have answered each one?",
        sql=sql_queries[1]
)

'0d9b503a9fd7e3f2d57d1786c462003b-sql'

In [16]:
vn.train(sql = sql_queries[2])

Question generated with sql: What is the total points and duration for each user's quiz session, ordered by points in descending order? 
Adding SQL...


'9edcee69edcbc8338de4bef97e55ec34-sql'

In [23]:
vn.train(question= "Who are the users with confirmed emails?",
         sql=sql_queries[3])

'b3a1d2b6f561005f98928c55071b2528-sql'

In [14]:
import warnings
warnings.filterwarnings("ignore") 

In [57]:
vn.generate_sql(question="Can you give me the users with confirmed emails from public.User? limiting result to 5")

SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE IF NOT EXISTS public."Answers" (\n    "Id" uuid PRIMARY KEY DEFAULT gen_random_uuid(),\n    "CreatedAt" timestamptz NOT NULL,\n    "LastUpdatedAt" timestamptz NOT NULL,\n    "OptionId" uuid NOT NULL REFERENCES public."Options" ("Id") ON DELETE CASCADE,\n    "QuestionId" uuid NOT NULL REFERENCES public."Questions" ("Id") ON DELETE CASCADE,\n    "QuizSessionId" uuid NOT NULL REFERENCES public."QuizSessions" ("Id") ON DELETE CASCADE,\n    "UserId" uuid NOT NULL REFERENCES public."Users" ("Id") ON DELETE CASCADE\n);\n\n\n\n===Additional Context \n\nThe following columns are in the Users table in the quizzard database:\n\n|    | table_catalog   | table_schema   | table_name   | column_name                      | data_

'SELECT \n    "FirstName",\n    "LastName",\n    "Email"\nFROM public."Users"\nWHERE "EmailConfirmed" = true\nLIMIT 5;'

In [48]:
train = vn.get_training_data()

In [25]:
train["content"].iloc[[109]]

109    The following columns are in the __EFMigration...
Name: content, dtype: object

In [ ]:
#vn.remove_training_data(id='2422154-doc')